To install pytorch geometric run the cell below

In [ ]:
#!pip install torch=='1.9.0'
#!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.9.0+cu102.html

# IMPORT

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score,average_precision_score

from torch_geometric.utils import negative_sampling
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges
from torch_geometric.transforms import RandomLinkSplit,NormalizeFeatures,Constant,OneHotDegree
from torch_geometric.utils import from_networkx
from torch_geometric.nn import GCNConv,SAGEConv,GATConv
from scipy.stats import entropy

import torch
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

import copy
import itertools
import json

# LOAD DATASET

You have to choose only one of the following four cells to construct the time slices over a certain time interval

In [ ]:
#time period used in the works
time_slices = {
    0: {
        'future_start': '2016-07-03',
        'future_end': '2016-08-02',
        'start' : '2016-06-03',
        'end': '2016-07-02'
    },
    
    1: {
        'future_start': '2016-08-03',
        'future_end': '2016-09-02',
        'start': '2016-07-03',
        'end': '2016-08-02'
    },
    
    2:{
        'future_start': '2016-09-03',
        'future_end': '2016-10-02',
        'start': '2016-08-03',
        'end': '2016-09-02'
    },
    
    3:{
        'future_start': '2016-10-03',
        'future_end': '2016-11-02',
        'start': '2016-09-03',
        'end': '2016-10-02'
    }
}

In [ ]:
#10 days time slice
time_slices = {
    0: {
        'future_start': '2016-08-03',
        'future_end': '2016-08-12',
        'start' : '2016-06-03',
        'end': '2016-08-02'
    },
    
    1: {
        'future_start': '2016-08-13',
        'future_end': '2016-08-22',
        'start': '2016-08-03',
        'end': '2016-08-12'
    },
    
    2:{
        'future_start': '2016-08-23',
        'future_end': '2016-09-01',
        'start': '2016-08-13',
        'end': '2016-08-22'
    },
    
    3:{
        'future_start': '2016-09-02',
        'future_end': '2016-09-11',
        'start': '2016-08-23',
        'end': '2016-09-01'
    },
    
    4:{
        'future_start': '2016-09-12',
        'future_end': '2016-09-21',
        'start': '2016-09-02',
        'end': '2016-09-11'
    },
    
}

In [ ]:
#Nov-Dec 2016 time interval
time_slices = {
    0: {
        'future_start': '2016-12-03',
        'future_end': '2017-01-02',
        'start' : '2016-06-03',
        'end': '2016-12-02'
    },
    
    1: {
        'future_start': '2017-01-03',
        'future_end': '2017-02-02',
        'start': '2016-12-03',
        'end': '2017-01-02'
    },
    
    2:{
        'future_start': '2017-02-03',
        'future_end': '2017-03-02',
        'start': '2017-01-03',
        'end': '2017-02-02'
    },
}

In [ ]:
#Nov-Dec 2016 time interval with 10 days time slice
time_slices = {
    0: {
        'future_start': '2016-12-03',
        'future_end': '2016-12-12',
        'start' : '2016-06-03',
        'end': '2016-12-02'
    },
    
    1: {
        'future_start': '2016-12-13',
        'future_end': '2016-12-22',
        'start': '2016-12-03',
        'end': '2016-12-12'
    },
    
    2:{
        'future_start': '2016-12-23',
        'future_end': '2017-01-01',
        'start': '2016-12-13',
        'end': '2016-12-22'
    },
    
    3:{
        'future_start': '2017-01-02',
        'future_end': '2017-01-11',
        'start': '2016-12-23',
        'end': '2017-01-01'
    },
    4:{
        'future_start': '2017-01-12',
        'future_end': '2017-01-21',
        'start': '2017-01-02',
        'end': '2017-01-11'
    },
}

In [ ]:
PERIOD = 1

start_train = time_slices[0]['start']
end_train = time_slices[PERIOD]['end']

start_future = time_slices[0]['start']
end_future = time_slices[PERIOD]['future_end']

start_future_test = time_slices[0]['start']
end_future_test = time_slices[PERIOD+1]['future_end']

In [ ]:
#gpath_train = f"dataset/graph/custom_json_{start_train}_{end_train}"
#gpath_future = f"dataset/graph/custom_json_{start_future}_{end_future}"
#gpath_test_future = f"dataset/graph/custom_json_{start_future_test}_{end_future_test}"
gpath_train = f"../dummy-data/gnn/custom_json_{start_train}_{end_train}"
gpath_future = f"../dummy-data/gnn/custom_json_{start_future}_{end_future}"
gpath_test_future = f"../dummy-data/gnn/custom_json_{start_future_test}_{end_future_test}"

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#textfpath_train = f"dataset/textFeatures/textFeatures_{start_train}_{end_train}.json" 
#textfpath_test = f"dataset/textFeatures/textFeatures_{start_future}_{end_future}.json"
textfpath_train = f"../dummy-data/gnn/textFeatures_{start_train}_{end_train}.json" 
textfpath_test = f"../dummy-data/gnn/textFeatures_{start_future}_{end_future}.json"

In [ ]:
"""
#TO SEE NEW EDGES

G_current_all = nx.read_edgelist(gpath_train, create_using=nx.DiGraph)
print('Number of edges {}'.format(G_current_all.size()))
G_future = nx.read_edgelist(gpath_future, create_using=nx.DiGraph)
k = len(G_future.subgraph(G_current_all.nodes()).edges())
print("New links {}".format(k))

"""

In [ ]:
def load_pyg_dataset(gpath_current, gpath_future, feature = 'struct', textfpath = None, topicSim=False):
    
    if feature not in ['constant','one-hot-encode','struct','text','topic','all']:
        raise Exception('Invalid feature value')
    
    #LOAD GRAPH AND COMPUTE NODE FEATURES
    G_current_all = nx.read_edgelist(gpath_current, create_using=nx.DiGraph)

    if feature == 'struct' or feature == 'all':
        #COMPUTE STRUCTURAL FEATURES
        pr = nx.pagerank(G_current_all)
        indeg = dict(G_current_all.in_degree())
        outdeg = dict(G_current_all.out_degree())
        neighdeg = nx.average_neighbor_degree(G_current_all)
    
    if feature == 'text' or feature == 'topic' or feature == 'all':
        #COMPUTE TEXTUAL FEATURES
        if textfpath is None:
            raise Exception('textfpath parameter must be not None if feature is text or all')
        with open(textfpath) as f:
            textf_raw = json.load(f)
            textf = dict()
            textf['numPost'] = textf_raw['numPost']
            textf['numComment'] = textf_raw['numComment']
            textf['numTag'] = textf_raw['numTag']
            textf['corpusLength'] = textf_raw['corpusLength']
            textf['corpusStdev'] = textf_raw['corpusStdev']
            
            if feature == 'topic':
                topics = {}
                for user,lda in textf_raw['lda_all'].items():
                    for i in range(30): #30 is the number of LDA topic
                        if f'topic_{i}' not in topics:
                            topics[f'topic_{i}'] = {}
                        topics[f'topic_{i}'][user] = lda[i]
                        
                for name,topicf in topics.items():
                    textf[name] = topicf
            
            if topicSim: 
                #compute an index based on entropy of LDA distributions to reflect
                #the user importance in term of its contents
                #the study on this feature is still not covered on our works
                entropy_all = {user:entropy(ldaAll) for user,ldaAll in textf_raw['lda_all'].items()}
                #entropy_post = {user:entropy(ldaPost) for user,ldaPost in textf_raw['lda_post'].items()}
                #entropy_comment = {user:entropy(ldaComment) for user,ldaComment in textf_raw['lda_comment'].items()}
                reciprocal_mean_followers_entropy_all = {}
                for user in G_current_all.nodes():
                    sum_followers_entropies = 0
                    count_followers = 0
                    for follower in G_current_all.predecessors(user):
                        follower_entropy = entropy_all[follower]
                        sum_followers_entropies += follower_entropy
                        count_followers += 1
                    mean_followers_entropy = 0
                    if count_followers > 0:
                        mean_followers_entropy = sum_followers_entropies / count_followers
                    reciprocal_mean = 0
                    if mean_followers_entropy != 0:
                        reciprocal_mean = 1 / mean_followers_entropy
                    reciprocal_mean_followers_entropy_all[user] = reciprocal_mean
                textf['reciprocal_mean_followers_entropy_all'] = reciprocal_mean_followers_entropy_all
                    
                
    #POSITIVE SET WITH LABELLING ON THE FUTURE PERIOD
    G_future = nx.read_edgelist(gpath_future, create_using=nx.DiGraph)

    G_current = nx.DiGraph(G_future.subgraph(G_current_all.nodes()))
    
    if feature == 'struct' or feature == 'text' or feature == 'topic' or feature == 'all':
        if feature == 'struct' or feature == 'all':
            #ASSIGN STRUCTURAL FEATURES TO EACH NODE
            nx.set_node_attributes(G_current,pr,'pagerank')
            nx.set_node_attributes(G_current,indeg,'in_degree')
            nx.set_node_attributes(G_current,outdeg,'out_degree')
            nx.set_node_attributes(G_current,neighdeg,'neigh_degree')
            #nx.set_node_attributes(G_train,nx.clustering(G_train),'clustering')
            #nx.set_node_attributes(G_train,nx.katz_centrality(G_train),'katz')
            
        if feature == 'text' or feature == 'all' or feature == 'topic':
            #ASSIGN TEXTUAL FEATURES TO EACH NODE
            for name, fdict in textf.items():
                nx.set_node_attributes(G_current, fdict, name)
    
    #CREATE PYG DATASET
    if feature != 'constant' and feature != 'one-hot-encode':
        group_attrs = all
    else:
        group_attrs = None

        
    current_data = from_networkx(G_current, group_node_attrs = group_attrs)
    current_data.train_mask = current_data.val_mask = current_data.test_mask = current_data.y = None
    
    if feature == 'constant':
        transform = Constant()
        current_data = transform(current_data)
    
    if feature == 'one-hot-encode':
        raise NotImplementedError('one-hot-encode not implemented')
        
    return current_data

In [ ]:
current_data = load_pyg_dataset(gpath_train, gpath_future, feature = 'topic', textfpath = textfpath_train)

#NORMALIZATION (L1-Norm)

transform = NormalizeFeatures()
current_data = transform(current_data)

#TRAIN TEST SPLIT + NEGATIVE SAMPLING
transform = RandomLinkSplit(num_val=0.0,num_test=0.25)
train_data, val_data, current_test_data = transform(current_data)

In [ ]:
future_data = load_pyg_dataset(gpath_future, gpath_test_future, feature = 'topic', textfpath = textfpath_test)

#NORMALIZATION
transform = NormalizeFeatures()
future_data = transform(future_data)

#NEGATIVE SAMPLING
future_neg_edge_index = negative_sampling(
        edge_index=future_data.edge_index, #positive edges
        num_nodes=future_data.num_nodes, # number of nodes
        num_neg_samples=future_data.edge_index.size(1)) # number of neg_sample equal to number of pos_edges

#edge index ok, edge_label concat, edge_label_index concat
num_pos_edge = future_data.edge_index.size(1)
future_data.edge_label = torch.Tensor(np.array([1 for i in range(num_pos_edge)] + [0 for i in range(num_pos_edge)]))
future_data.edge_label_index = torch.cat([future_data.edge_index, future_neg_edge_index], dim=-1)

In [ ]:
train_data

In [ ]:
future_data

In [ ]:
#MY GNN CUSTOM MODULE
class LinkPredModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, layer = GCNConv, dropout=0.25, loss = torch.nn.BCEWithLogitsLoss):
        
        super(LinkPredModel, self).__init__()
        self.conv1 = layer(input_dim, hidden_dim)
        self.conv2 = layer(hidden_dim, num_classes)
        
        #Initialize the loss function to BCEWithLogitsLoss
        self.loss_fn = loss()

        self.dropout = dropout

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, batch):
        x , edge_index, edge_label_index = batch.x.float(), batch.edge_index, batch.edge_label_index
        
        ## Note
        ## 1. Feed the node feature into the first conv layer
        ## 2. Add a leaky-ReLU after the first conv layer
        ## 3. Add dropout after the ReLU (with probability self.dropout)
        ## 4. Repeat for the next layers
        ## 5. Select the embeddings of the source nodes and destination nodes
        ## by using the edge_label_index and compute the similarity of each pair
        ## by dot product

        h = self.conv1(x, batch.edge_index)
        h = F.leaky_relu(h)
        h = F.dropout(h, p=self.dropout)
        h = self.conv2(h, edge_index)
        h = F.leaky_relu(h)
        h = F.dropout(h, p=self.dropout)
        
        h_src = h[edge_label_index[0]]
        h_dst = h[edge_label_index[1]]
        h_sim = h_src * h_dst #dot product
        pred = torch.sum(h_sim, dim=-1)

        return pred
    
    def loss(self, pred, link_label):
        return self.loss_fn(pred, link_label)

In [ ]:
from sklearn.metrics import *

def train(model, train_data, val_data, test_data, device,\
          optimizer, num_epochs=200, verbose=True):
    
    avgpr_val_max = 0
    best_model = copy.deepcopy(model)
    train_data = train_data.to(device)
    best_epoch = -1
    
    avgpr_trains = []
    avgpr_vals = []
    avgpr_tests = []
    
    #roc_trains = []
    #roc_vals = []
    #roc_tests = []
    
    for epoch in range(num_epochs):
        model.train()
        ## Note
        ## 1. Zero grad the optimizer
        ## 2. Compute loss and backpropagate
        ## 3. Update the model parameters
        optimizer.zero_grad()
            
        #pred = best_model(train_data)

        pred = model(train_data)
        loss = model.loss(pred, train_data.edge_label.type_as(pred))

        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.

        ##########################################

        log = 'Epoch: {:03d}\n AVGPR Train: {:.4f}, Val: {:.4f}, Test: {:.4f}\n ROC Train: {:.4f}, Val: {:.4f}, Test: {:.4f}\n F1-Score Train: {:.4f}, Val: {:.4f}, Test: {:.4f}\n Loss: {}'
        avgpr_score_train, f1_score_train, roc_score_train = test(model, train_data, device)
        avgpr_score_val, f1_score_val, roc_score_val = test(model, val_data, device)
        avgpr_score_test, f1_score_test, roc_score_test = test(model, test_data, device)
        #score_test = test(model, dataloaders['test'], args)
        
        #f1_trains.append(f1_score_train)
        #f1_vals.append(f1_score_val)
        #f1_tests.append(f1_score_test)
        
        avgpr_trains.append(avgpr_score_train)
        avgpr_vals.append(avgpr_score_val)
        avgpr_tests.append(avgpr_score_test)
        
        if verbose:
            print(log.format(epoch, avgpr_score_train, avgpr_score_val, avgpr_score_test, roc_score_train, roc_score_val, roc_score_test, f1_score_train, f1_score_val, f1_score_test, loss.item()))
            
        if avgpr_val_max < avgpr_score_val:
            avgpr_val_max = avgpr_score_val
            best_epoch = epoch
            best_model = copy.deepcopy(model)
            
    if verbose:
        print(f'Best Epoch: {best_epoch}')
        
    return best_model, avgpr_trains, avgpr_vals, avgpr_tests

In [ ]:
def test(model, test_data, device):
    model.eval()
    
    f1_model_score=0

    test_data = test_data.to(device)

    h = model(test_data)
    
    pred_cont = torch.sigmoid(h).cpu().detach().numpy()
    pred = [1 if p > 0.5 else 0 for p in pred_cont]

    label = test_data.edge_label.cpu().detach().numpy()
      
    roc_score = roc_auc_score(label, pred_cont)
    avgpr_score = average_precision_score(label, pred_cont)
    f1_model_score = f1_score(label,pred)
 
    return avgpr_score, f1_model_score, roc_score

In [ ]:
input_dim = train_data.num_node_features
num_classes = 2

## Tuning

Run this part of the notebook only if you want to re-perform hyperparameter tuning. Below you will find the model with the best config of hypeparameters found.

In [ ]:
grid_all = {
    'optimizer': ['sgd','adam','adagrad'],
    'hidden_dim': [4,8,16, 32, 64, 128, 256],
    'layer': [GATConv, GCNConv, SAGEConv],
    'dropout': [0,0.25,0.50,0.75],
    'weight_decay': [5e-5,5e-3,5e-2,5e-1,5e-6],
    'lr': [1,0.1,0.075,0.05,0.025,0.01,0.001,0.0001,0.00001]
}

In [ ]:
grid_1 = {
    'optimizer': ['sgd','adam','adagrad'],
    'hidden_dim': [16, 32, 64, 128, 256],
    'layer': [GATConv, GCNConv, SAGEConv],
    'dropout': [0.25,0.50,0.75],
    'lr': [1,0.1,0.01,0.001,0.0001,0.00001]
}

## USATA PER PRIMI EXP (STRUCT F PERIOD 1)

In [ ]:
grid_2 = {
    'optimizer': ['sgd','adam','adagrad'],
    'hidden_dim': [16,32],
    'layer': [GATConv, GCNConv, SAGEConv],
    'dropout': [0.0, 0.25],
    'lr': [0.1,0.01,0.001]
}

## USATA PER CONSTANT PERIOD 1

In [ ]:
grid_3 = {
    'optimizer': ['adam'],
    'hidden_dim': [16,32],
    'layer': [GCNConv],
    'dropout': [0.0],
    'lr': [0.1,0.01]
}

In [ ]:
grid_4 = {
    'optimizer': ['adam'],
    'hidden_dim': [4,8],
    'layer': [GCNConv],
    'dropout': [0.0],
    'lr': [0.025,0.05,0.075],
    'weight_decay': [5e-5,5e-3,5e-2,5e-1,5e-6]
    
}

In [ ]:
def gnn_tuning(grid, input_dim, num_classes, device, train_data, current_test_data, test_data, logname, fold, plotF1=True):
    names, values = zip(*grid.items())
    configurations = [dict(zip(names,v)) for v in itertools.product(*values)]
    print('There are ', len(configurations), ' configurations for this model')
    
    i=0
    avgpr_max = 0
    best_config = {}
    best_config_i = 0
    weight_decay = 5e-4
    with open(logname,'a+') as log:
        for config in configurations:
            print('Configuration number ', str(i), ' start')
            
            log.write('CONFIGURATION ' + str(i) + '\n\n')
            writeConfig = copy.deepcopy(config)
            writeConfig['layer'] = writeConfig['layer'].__name__
            nameLayer = writeConfig['layer']
            log.write(json.dumps(writeConfig,indent=2))
            log.write('\n')

            hidden_dim = config['hidden_dim']
            layer = config['layer']
            dropout = config['dropout']
            if 'weight_decay' in config:
                weight_decay = config['weight_decay']
            model = LinkPredModel(input_dim, hidden_dim, num_classes, dropout = dropout, layer = layer).to(device)
            model.reset_parameters()
            
            opt = config['optimizer']
            lr = config['lr']
            if opt == 'sgd':
                optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
            elif opt == 'adam':
                optimizer = torch.optim.Adam(params=model.parameters(), lr=lr, weight_decay = weight_decay)
            else:
                optimizer = torch.optim.Adagrad(model.parameters(), lr=lr, weight_decay=weight_decay)

            best_model, avgpr_trains, avgpr_vals, avgpr_tests = \
            train(model, train_data, current_test_data, test_data, device, optimizer,verbose=False)
            
            print('GNN fitted')

            avgpr_train, f1_score_train, roc_score_train = test(best_model, train_data, device)
            avgpr_val, f1_score_current_test, roc_score_val = test(best_model, current_test_data, device)
            avgpr_test, f1_score_future_test, roc_score_test = test(best_model, future_data, device)
            
            print('GNN performance computed')

            log.write('TRAIN SET\n')
            log.write(f' AVGPR SCORE: {avgpr_train}\n')
            log.write(f' ROC SCORE: {roc_score_train}\n')
            log.write(f' F1-SCORE: {f1_score_train}\n\n')

            log.write('CURRENT TEST SET\n')
            log.write(f' AVGPR SCORE: {avgpr_val}\n')
            log.write(f' ROC SCORE: {roc_score_val}\n')
            log.write(f' F1-SCORE: {f1_score_current_test}\n\n')

            log.write('FUTURE TEST SET\n')
            log.write(f' AVGPR SCORE: {avgpr_test}\n')
            log.write(f' ROC SCORE: {roc_score_test}\n')
            log.write(f' F1-SCORE: {f1_score_future_test}\n')
            
            print('GNN performance written')

            if avgpr_max < avgpr_test:
                avgpr_max = avgpr_test
                best_config = config
                best_config_i = i
                scores = {
                    'avgpr_train': avgpr_train,
                    'roc_score_train': roc_score_train,
                    'f1_score_train': f1_score_train,
                    'avgpr_current_test': avgpr_val,
                    'roc_score_val': roc_score_val,
                    'f1_score_current_test': f1_score_current_test,
                    'avgpr_future_test': avgpr_test,
                    'roc_score_test': roc_score_test,
                    'f1_score_future_test': f1_score_future_test
                }
                top_model = copy.deepcopy(best_model)
                
            if plotF1:
                #train orange test blue val green
                num_epochs = 200
                x = range(num_epochs)
                plt.clf()
                plt.plot(x, avgpr_trains, color='orange', label='avgpr_train')
                plt.plot(x, avgpr_vals, color='green', label='avgpr_val')
                plt.plot(x, avgpr_tests, color='blue', label = 'avgpr_test')
                plt.xlabel('Epoch')
                plt.ylabel('AVGPR-score')
                plt.legend()
                plt.ylim(top=1)
                plt.grid()
                plt.savefig(f'learningCurves/{fold}/{nameLayer}_{hidden_dim}_{dropout}_{opt}_{lr}_{weight_decay}.pdf'\
                            ,bbox_inches='tight')
                plt.clf()
            
            print('Configuration number ', str(i), ' end')
            print()
            
            log.write('\n\n')
            log.flush()

            i+=1
            
        log.write(f'Best configuration number: {best_config_i}\n')
        best_config_write = copy.deepcopy(best_config)
        best_config_write['layer'] = best_config_write['layer'].__name__
        log.write(json.dumps(best_config_write,indent=2))
        log.write('\n')
        log.write(json.dumps(scores,indent=2))
        log.write('\n')
        log.flush()
        log.close()
        
    return top_model, best_config, scores

In [ ]:
best_model, best_config, scores = gnn_tuning(grid_4, input_dim, num_classes, device, train_data, current_test_data, future_data, 'new_grid_gnn_text_august2016.txt', 'GNN/new_grid/text')

## Single Train

In [ ]:
model = LinkPredModel(input_dim, 16, num_classes, dropout=0.0, layer=GCNConv).to(device)
model.reset_parameters()

In [ ]:
weight_decay = 5e-4
#optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=weight_decay)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01, weight_decay = weight_decay)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1, weight_decay = weight_decay)
#optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01, weight_decay=weight_decay)

In [ ]:
best_model , prtrain , prval, prtest = train(model, train_data, current_test_data, future_data, device, optimizer)

In [ ]:
#train orange test blue val green
num_epochs = 200
x = range(num_epochs)
plt.clf()
plt.plot(x, prtrain, color='orange', label='avgpr_train')
plt.plot(x, prval, color='green', label='avgpr_val')
plt.plot(x, prtest, color='blue', label = 'avgpr_test')
plt.xlabel('Epoch')
plt.ylabel('AVGPR-score')
plt.legend()
plt.ylim(top=1)
plt.grid()
plt.savefig(f'new_AVGPR_learningCurve_GNNAll_august2016.pdf',bbox_inches='tight')
plt.show()
plt.clf()

In [ ]:
roc_train, f1_score_train = test(best_model, train_data, device)
roc_val, f1_score_current_test = test(best_model, current_test_data, device)
roc_test, f1_score_future_test = test(best_model, future_data, device)

In [ ]:
print('TRAIN SET')
print(f' ROC AUC SCORE: {roc_train}')
print(f' F1-SCORE: {f1_score_train}')

print()

print('CURRENT TEST SET')
print(f' ROC AUC SCORE: {roc_val}')
print(f' F1-SCORE: {f1_score_current_test}')

print()

print('FUTURE TEST SET')
print(f' ROC AUC SCORE: {roc_test}')
print(f' F1-SCORE: {f1_score_future_test}')